In [16]:
# .env 파일에 OPENAI_API_KEY=<YOUR_KEY> 형식으로 API 키를 넣어 두었다고 가정합니다.
from dotenv import load_dotenv
load_dotenv()  # .env 파일의 환경변수를 로드합니다.

import os
openai_api_key = os.getenv('OPENAI_API_KEY') 
# OpenAI API 키가 정상적으로 불러와졌는지 확인 (키 문자열 일부 출력)
print(openai_api_key[:8] + "****") if openai_api_key else print("API Key not found!")

sk-proj-****


In [17]:
from langchain_openai import ChatOpenAI
# OpenAI GPT-4o-mini 모델 초기화 (temperature 등의 파라미터 조정 가능)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

In [18]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. 프롬프트 템플릿 정의 (한국어 프롬프트)
product_prompt = PromptTemplate.from_template(
    "제품 이름: {product}\n" + 
    "이 제품의 특징과 장점을 매력적인 한 단락으로 설명해주세요."
)

# 2. 출력 파서 정의 (문자열 출력 파서)
output_parser = StrOutputParser()

# 3. LCEL 체인 구성: 프롬프트 → LLM → 출력파서
product_chain = product_prompt | llm | output_parser

# 4. 체인 실행 (.invoke) - 예시 입력으로 실행
example_input = {"product": "무선 블루투스 이어폰"}  # 예시로 "무선 블루투스 이어폰" 설명 생성
result = product_chain.invoke(example_input)

print(result)

무선 블루투스 이어폰은 탁월한 편안함과 자유로운 이동성을 제공하여, 언제 어디서나 음악과 통화를 즐길 수 있는 완벽한 동반자입니다. 최신 블루투스 기술을 적용하여 안정적인 연결을 보장하며, 고음질 사운드와 깊은 베이스로 여러분의 음악 경험을 한층 풍부하게 만듭니다. 인체공학적 디자인으로 제작되어 장시간 착용해도 불편함 없이 사용할 수 있으며, 한 번의 충전으로 긴 배터리 수명을 자랑합니다. 또한, 간편한 터치 컨트롤로 전화 수신과 음악 재생을 손쉽게 조작할 수 있어, 바쁜 일상 속에서도 스마트한 라이프스타일을 지원합니다. 무선 블루투스 이어폰으로 자유롭고 몰입감 있는 오디오 경험을 만끽해보세요!


In [19]:
from langchain_core.runnables import RunnablePassthrough

# 1. 이메일 제목 생성용 프롬프트 정의
subject_prompt = PromptTemplate.from_template(
    "다음 요청 내용을 바탕으로 이메일 제목을 지어주세요:\n{content}"
)

# 2. 이메일 본문 생성용 프롬프트 정의
body_prompt = PromptTemplate.from_template(
    "위에서 생성된 제목을 활용하여, 팀에게 보내는 정중한 이메일 본문을 작성해주세요.\n" + 
    "제목: {subject}\n본문:"
)

# 3. 두 프롬프트를 LLM과 결합한 체인 구성
email_chain = (
    subject_prompt          # 1단계: 제목 프롬프트 
    | llm                   # LLM 호출 -> 제목 생성
    | {"subject": RunnablePassthrough()}   # 출력된 제목을 subject 키로 매핑
    | body_prompt           # 2단계: 본문 프롬프트 (subject를 입력으로 받음)
    | llm                   # LLM 호출 -> 본문 생성
)

# 4. 체인 실행 예시
email_request = {
    "content": "다음주에 프로젝트 진행 상황을 논의하기 위해 팀 회의를 요청드리는 이메일"
}
final_email = email_chain.invoke(email_request)
print(final_email)

content='안녕하세요, 팀 여러분.\n\n다음 주에 진행될 팀 회의에 대해 말씀드리고자 합니다. 이번 회의에서는 현재 진행 중인 프로젝트의 상황을 논의할 예정입니다. 각자 맡고 있는 업무의 진행 상황과 앞으로의 계획을 공유하는 시간을 가지면 좋겠습니다.\n\n회의 일정은 다음과 같습니다:\n- 날짜: [날짜 입력]\n- 시간: [시간 입력]\n- 장소: [장소 입력] / 온라인 링크: [링크 입력]\n\n모두의 참여가 중요하니, 가능한 많은 팀원들이 참석해 주시기를 부탁드립니다. 회의에서 다루고 싶은 추가적인 내용이 있으시면 미리 알려주시면 좋겠습니다.\n\n감사합니다.\n\n[당신의 이름]  \n[당신의 직책]  \n[회사 이름]  ' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 169, 'prompt_tokens': 319, 'total_tokens': 488, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BauRQRknwFD678aisxLbPoo4zyaYW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--2ca0f740-2ffc-429f-8e00-383feadd6da0-0' usage_metadata={'input_tokens': 319, 'output

In [20]:
from langchain_core.runnables import RunnableBranch, RunnableLambda

# 1. 요약 체인 정의 (프롬프트 → LLM)
summary_prompt = PromptTemplate.from_template("다음 글을 한 문단으로 요약해주세요:\n{text}")
summary_chain = summary_prompt | llm  # (StrOutputParser를 쓰지 않아도 기본 문자열 출력)

# 2. 이메일 체인은 이전 실습의 email_chain 활용 (이미 정의되었다고 가정; 없다면 위에서 정의한 코드 재사용)

# 3. 분기 조건 함수 정의 및 Runnable로 래핑
def is_summary_request(user_input: str) -> bool:
    return user_input.strip().startswith("요약:")  # 입력이 "요약:"으로 시작하면 True

condition = RunnableLambda(is_summary_request)

# 4. 분기 체인 구성: (조건, 조건=True일 때 체인)와 False일 때 체인 지정
branch_chain = RunnableBranch(
    (condition, summary_chain),  # 조건이 True이면 summary_chain 실행
    email_chain                 # 조건 False이면 email_chain 실행
)

# 5. 다양한 입력에 대해 분기 체인 실행
input1 = "요약: 어제 진행된 회의에서는 프로젝트 일정 변경과 예산 조정에 대한 논의가 있었습니다."
result1 = branch_chain.invoke(input1)
print("[요약 요청 결과]\n", result1, "\n")

input2 = "이메일: 다음 주 월요일 프로젝트 회의 일정을 팀에 공지해줘."
result2 = branch_chain.invoke(input2)
print("[이메일 작성 요청 결과]\n", result2)

[요약 요청 결과]
 content='어제 회의에서는 프로젝트 일정 변경과 예산 조정에 대해 논의되었습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 45, 'total_tokens': 64, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BauRURsJqtb0Iqg1NzTlg6FclShXB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--a7f6cb46-28e6-455d-a997-d08faa2ad181-0' usage_metadata={'input_tokens': 45, 'output_tokens': 19, 'total_tokens': 64, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}} 

[이메일 작성 요청 결과]
 content='제목: 다음 주 월요일 프로젝트 회의 일정 공지\n\n안녕하세요, 팀 여러분.\n\n다음 주 월요일에 예정된 프로젝트 회의 일정에 대해 안내드립니다. 이번 회의에서는 현재 진행 중인 프로젝트의 진행 상황과 앞으로의 계획에 대해 논의할 예정입니다.

In [21]:
# .env 파일 로드
from dotenv import load_dotenv
import os

load_dotenv()  # .env 파일의 환경변수 로드
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OWM_API_KEY = os.getenv("OPENWEATHERMAP_API_KEY")

# 키 값 확인 (None이 아니라면 성공적으로 불러온 것입니다)
print("OpenAI Key present?", OPENAI_API_KEY is not None)
print("Weather API Key present?", OWM_API_KEY is not None)

OpenAI Key present? True
Weather API Key present? True


In [22]:
import requests
from pydantic import BaseModel, Field

# 입력 스키마 정의
class WeatherInput(BaseModel):
    city: str = Field(description="날씨를 조회할 도시 이름 (영문)")

def get_weather(city: str) -> str:
    """주어진 도시의 현재 날씨를 반환합니다."""
    api_key = os.getenv("OPENWEATHERMAP_API_KEY")
    url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": api_key,
        "units": "metric",      # 섭씨 온도
        "lang": "kr"            # 한국어 응답 (설명이 한국어로 오도록)
    }
    response = requests.get(url, params=params)
    data = response.json()
    # API 응답에 따른 처리
    if data.get("cod") != 200:
        # 도시 정보를 찾지 못한 경우
        return f"'{city}'의 날씨 정보를 찾을 수 없습니다."
    # 필요한 정보 파싱
    temp = data["main"]["temp"]
    desc = data["weather"][0]["description"]
    city_name = data["name"]  # 응답에 있는 도시 이름 (영문일 수 있음)
    return f"{city_name}의 현재 기온은 {temp}℃, 날씨는 {desc}입니다."

In [23]:
get_weather('Seoul')

'Seoul의 현재 기온은 19.76℃, 날씨는 맑음입니다.'

In [24]:
# 입력 스키마 정의
class NewsInput(BaseModel):
    keyword: str = Field(description="최신 뉴스를 검색할 키워드 (한글 또는 영문)")

def get_news(keyword: str) -> str:
    """주어진 키워드에 대한 최신 뉴스 제목을 반환합니다."""
    api_key = os.getenv("NEWSAPI_API_KEY")
    url = "https://newsapi.org/v2/everything"
    params = {
        "q": keyword,
        "language": "ko",         # 한국어 뉴스 기사 검색
        "sortBy": "publishedAt",  # 최신 뉴스 우선 정렬
        "pageSize": 1,            # 가장 최근 기사 1건만 가져오기
        "apiKey": api_key
    }
    res = requests.get(url, params=params)
    data = res.json()
    articles = data.get("articles")
    if not articles:
        return f"'{keyword}'에 대한 최신 뉴스가 없습니다."
    # 가장 첫 번째 뉴스를 선택
    top_article = articles[0]
    title = top_article.get("title", "(제목 없음)")
    source = top_article.get("source", {}).get("name", "")
    return f"'{keyword}' 관련 뉴스: {title} - {source}"

In [25]:
print(get_news("OpenAI"))

'OpenAI' 관련 뉴스: AI가 AI를 평가했더니…"도움 드릴 수 없습니다" 답변을 32% 더 선호 - Zdnet.co.kr


In [26]:
from langchain_core.runnables import RunnableLambda

# 날씨 함수 runnable -> tool 변환
weather_runnable = RunnableLambda(lambda input: get_weather(input["city"]))
weather_tool = weather_runnable.as_tool(
    name="get_weather",
    description="도시 이름(영문)을 입력하면 현재 날씨를 알려주는 도구",
    args_schema=WeatherInput
)

# 뉴스 함수 runnable -> tool 변환
news_runnable = RunnableLambda(lambda input: get_news(input["keyword"]))
news_tool = news_runnable.as_tool(
    name="get_news",
    description="키워드를 입력하면 관련된 최신 뉴스 제목을 알려주는 도구",
    args_schema=NewsInput
)

print(f"Tool 이름: {weather_tool.name}, 설명: {weather_tool.description}")
print(f"Tool 이름: {news_tool.name}, 설명: {news_tool.description}")

Tool 이름: get_weather, 설명: 도시 이름(영문)을 입력하면 현재 날씨를 알려주는 도구
Tool 이름: get_news, 설명: 키워드를 입력하면 관련된 최신 뉴스 제목을 알려주는 도구


In [27]:
from langchain.agents import initialize_agent, AgentType

# 앞서 만든 Tool 객체 리스트 준비
tools = [weather_tool, news_tool]

# OpenAI 함수 기반 에이전트 생성
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True  # intermediate 단계 로그를 출력하여 확인
)

In [29]:
query1 = "서울의 날씨와 AI 관련 최신 뉴스를 알려줘."
result1 = agent.run(query1)
print("Agent Result 1:", result1)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'Seoul'}`


Seoul의 현재 기온은 19.76℃, 날씨는 맑음입니다.
Invoking: `get_news` with `{'keyword': 'AI'}`


'AI' 관련 뉴스: AI 시대 B2B 비즈니스 그리고 투자, 언바운드랩 조용민 대표 [B2B Roadmap 2025 Day 2 : Session 1] - Venturesquare.net현재 서울의 날씨는 맑으며, 기온은 19.76℃입니다.

AI 관련 최신 뉴스는 다음과 같습니다:
- **AI 시대 B2B 비즈니스 그리고 투자, 언바운드랩 조용민 대표** [B2B Roadmap 2025 Day 2 : Session 1] - Venturesquare.net

더 궁금한 점이 있으면 말씀해 주세요!

> Finished chain.
Agent Result 1: 현재 서울의 날씨는 맑으며, 기온은 19.76℃입니다.

AI 관련 최신 뉴스는 다음과 같습니다:
- **AI 시대 B2B 비즈니스 그리고 투자, 언바운드랩 조용민 대표** [B2B Roadmap 2025 Day 2 : Session 1] - Venturesquare.net

더 궁금한 점이 있으면 말씀해 주세요!
